## Exercise 1

Poisson regression is a Generalized Linear Model, used to model count data. It takes the form

$$\mathbb{E}(\mu|x)=\exp(w_1\,x_1+\ldots+w_k\,x_k+b),$$

where the observed counts $y$ are drawn from a Poisson distribution on the expected counts: 

$$y_i \sim \text{Poisson}(\mu_i).$$

1. Download and import Load the smoking dataset from: [https://data.princeton.edu/wws509/datasets/#smoking](https://data.princeton.edu/wws509/datasets/#smoking). Then perform a train-test split on the data;


In [22]:
import numpy as np
import pandas as pd
import pyro
import pyro.distributions as dist
import pyro.optim as optim
from pyro.infer import SVI, Trace_ELBO
from pyro.infer import Predictive
import torch
import torch.distributions.constraints as constraints
from sklearn.model_selection import train_test_split
import seaborn as sns

sns.set_style("darkgrid")
figsize=(10,4)
pyro.set_rng_seed(42)

In [2]:
data = pd.read_csv("https://data.princeton.edu/wws509/datasets/smoking.raw",sep="\t",header=None)
data.columns = ["Age_Group","Smoking_Status","Population","Deaths"]
data.head()

,Age_Group,Smoking_Status,Population,Deaths
0,1,1,656,18
1,2,1,359,22
2,3,1,249,19
3,4,1,632,55
4,5,1,1067,117


* age at the start of follow-up: in five-year age groups coded 1 to 9 for 40-44, 45-49, 50-54, 55-59, 60-64, 65-69, 70-74, 75-79, 80+.
* smoking status: coded 1 = never smoked, 2 = smoked cigars or pipe only, 3 = smoked cigarettes and cigar or pipe, and 4 = smoked cigarettes only,
* population: number of male pensioners followed, and
* deaths: number of deaths in a six-year period.

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   Age_Group       36 non-null     int64
 1   Smoking_Status  36 non-null     int64
 2   Population      36 non-null     int64
 3   Deaths          36 non-null     int64
dtypes: int64(4)
memory usage: 1.2 KB


Now we perform a train-test split on the data:
- **train data** - 80% of the observations will be used to perfom inference on our model
- **test data** - the remaining 20% will be used for testing the correctness of posterior predictions 

In [42]:
deaths = torch.tensor(data["Deaths"].values, dtype=torch.int)
predictors = torch.stack([torch.tensor(data[column].values,dtype=torch.int) 
                            for column in data.columns if column != "Deaths"],1)

X_train, X_test, y_train, y_test = train_test_split(predictors, deaths, test_size=0.20, 
                                                    random_state=42,shuffle=True)

print("X_train.shape =", X_train.shape,"\ny_train.shape =", y_train.shape)
print("\nX_test.shape =", X_test.shape,"\ny_test.shape =", y_test.shape)

X_train.shape = torch.Size([28, 3]) 
y_train.shape = torch.Size([28])

X_test.shape = torch.Size([8, 3]) 
y_test.shape = torch.Size([8])


2. Fit a Poisson bayesian regression model using the number of deaths as the response variable and the other columns as the explanatory variables;

The target variable in our model is the number of deaths and we wish to infer the parameters corresponding to the following predictors

$$
\text{Deaths}=\text{exp}(w_0\cdot\text{Age_Group}+w_1\cdot\text{Smoking_Status}+w_2\cdot\text{Population}+b) + \epsilon
$$

We set a normal prior on $w$, a Log-Normal on the bias term $b$ and a uniformly distributed std for the gaussian noise on $\hat{y}$

\begin{align*}
w&\sim\mathcal{N}(0,1)\\
b&\sim\text{LogNormal}(0,1)\\
\hat{\mu}&= \text{exp}(w x+ b) \\
y &\sim \text{Poisson}(\hat{\mu}).
\end{align*}

Then we define the family of posterior distributions, by setting a Gamma distribution on $w$ and a Log-Normal on $b$, and run SVI inference on $(x,y)$ data.

Notice the prior distribution on the bias term makes this regression problem analytically intractable.

**will be read**

https://towardsdatascience.com/an-illustrated-guide-to-the-poisson-regression-model-50cccba15958

https://ncss-wpengine.netdna-ssl.com/wp-content/themes/ncss/pdf/Procedures/NCSS/Poisson_Regression.pdf

In [45]:
def death_model(predictors, deaths):
    
    n_observations, n_predictors = predictors.shape
    
    # sample weights
    w = pyro.sample("w", dist.Normal(torch.zeros(n_predictors), 
                                        torch.ones(n_predictors)))
    b = pyro.sample("b", dist.LogNormal(torch.zeros(1), torch.ones(1)))
    
    mu_hat = torch.exp((w*predictors).sum(dim=1) + b)
    
    # condition on the observations
    with pyro.plate("deaths", len(deaths)):
        pyro.sample("obs", dist.Poisson(mu_hat), obs=deaths)
        
def death_guide(predictors, deaths=None):
    
    n_observations, n_predictors = predictors.shape
        
    w_loc = pyro.param("w_loc", torch.rand(n_predictors), constraint=constraints.positive)
    w_scale = pyro.param("w_scale", torch.rand(n_predictors), 
                         constraint=constraints.positive)
    
    w = pyro.sample("w", dist.Gamma(w_loc, w_scale))
    
    b_loc = pyro.param("b_loc", torch.rand(1))
    b_scale = pyro.param("b_scale", torch.rand(1), constraint=constraints.positive)
    
    b = pyro.sample("b", dist.LogNormal(b_loc, b_scale))
    
death_svi = SVI(model=death_model, guide=death_guide, 
              optim=optim.ClippedAdam({'lr' : 0.01}), 
              loss=Trace_ELBO())

for step in range(2000):
    loss = death_svi.step(X_train, y_train)/len(X_train)
    if step % 100 == 0:
        print(f"Step {step} : loss = {loss}")


In [53]:
print("Inferred params:", list(pyro.get_param_store().keys()), end="\n\n")

# w_i and b posterior mean
inferred_w = pyro.get_param_store()["w_loc"]
inferred_b = pyro.get_param_store()["b_loc"]

for i,w in enumerate(inferred_w):
    print(f"w_{i} = {w.item():.8f}")
print(f"b = {inferred_b.item():.8f}")

Inferred params: ['w_loc', 'w_scale', 'b_loc', 'b_scale']

w_0 = 1.23886657
w_1 = 0.75426805
w_2 = 0.51143640
b = 1.39225626


**Posterior predictive distribution**

We can use the `Predictive` utility class, corresponding to the posterior predictive distribution, to evaluate our model on test data. Here we compute some summary statistics (mean, std and qualtiles) on $100$ samples from the posterior predictive:

In [54]:
# print latent params quantile information
def summary(samples):
    stats = {}
    for par_name, values in samples.items():
        marginal = pd.DataFrame(values)
        percentiles=[.05, 0.5, 0.95]
        describe = marginal.describe(percentiles).transpose()
        stats[par_name] = describe[["mean", "std", "5%", "50%", "95%"]]
    return stats

# define the posterior predictive
predictive = Predictive(model=death_model, guide=death_guide, num_samples=100,
                        return_sites=("w","b"))

# get posterior samples on test data
svi_samples = {k: v.detach().numpy() for k, v in predictive(X_test, y_test).items()}

# show summary statistics
for key, value in summary(svi_samples).items():
    print(f"Sampled parameter = {key}\n\n{value}\n")

Sampled parameter = w

       mean       std        5%       50%       95%
0  0.104522  0.085365  0.013744  0.076307  0.264744
1  0.058518  0.075987  0.001701  0.030012  0.230383
2  0.000975  0.001058  0.000011  0.000594  0.003477

Sampled parameter = b

     mean       std       5%       50%       95%
0  4.0254  0.046407  3.94176  4.028924  4.095379




3. Evaluate the regression fit on test data using MAE and MSE error metrics.

The most known metrics for comparing different regression models are the **Mean Absolute Error** (MAE)

$$\frac{1}{n}\sum_{i=1}^n |y_i - \hat{y}_i|$$

and the **Mean Squared Error** (MSE)

$$\frac{1}{n}\sum_{i=1}^n (y_i - \hat{y}_i)^2,$$

where $n$ is the number of observations, $y$ are the true values `y_test` and $\hat{y}$ are the predicted values `y_pred`.

In [55]:
# compute predictions using the inferred paramters
y_pred = torch.exp((inferred_w * X_test).sum(1)) + inferred_b

print("MAE =", torch.nn.L1Loss()(y_test, y_pred).item())
print("MSE =", torch.nn.MSELoss()(y_test, y_pred).item())

MAE = inf
MSE = inf


In [28]:
data.describe()

,Age_Group,Smoking_Status,Population,Deaths
count,36.000000,36.000000,36.000000,36.000000
mean,5.000000,2.500000,1558.944444,253.611111
std,2.618615,1.133893,1562.232175,262.597495
min,1.000000,1.000000,98.000000,2.000000
25%,3.000000,1.750000,369.250000,100.500000
50%,5.000000,2.500000,858.500000,171.500000
75%,7.000000,3.250000,2305.500000,274.000000
max,9.000000,4.000000,6052.000000,1001.000000


## Exercise 2

The Iris dataset contains petal and sepal length and width for three different types of Iris flowers: Setosa, Versicolour, and Virginica.

1. Import the Iris dataset from `sklearn`:
```
from sklearn import datasets
iris = datasets.load_iris()
```
and perform a train-test split on the data.

2. Fit a multinomial bayesian logistic regression model on the four predictors petal length/width and sepal length/width. 

3. Evaluate your bayesian classifier on test data: compute the overall test accuracy and class-wise accuracy for the three different flower categories.